In [ ]:
import requests
import pandas as pd
import sqlite3
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para criar a tabela no banco de dados SQLite (se ainda não existir)
def criar_tabela():
    with sqlite3.connect('dados_pontos.db') as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS onibus (
                NOME TEXT,
                NUM TEXT,
                Latitude REAL,
                Longitude REAL,
                SEQ INTEGER,
                GRUPO TEXT,
                SENTIDO TEXT,
                TIPO TEXT,
                ID_do_Itinerario INTEGER,
                COD TEXT
            )
        ''')
        conn.commit()

# Função para buscar e processar dados da linha de ônibus
def buscar_e_processar_dados(linha):
    url = f'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={linha:03}&c=821f0'
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        dados_json = response.json()

        # Verificar se os dados estão vazios
        if not dados_json:
            return None

        # Criar DataFrame a partir dos dados
        df = pd.DataFrame.from_records(dados_json)

        # Substituir vírgulas por pontos e converter tipos
        df['Latitude'] = df['LAT'].str.replace(',', '.').astype(float)
        df['Longitude'] = df['LON'].str.replace(',', '.').astype(float)
        df['SEQ'] = df['SEQ'].astype(int)
        df['ID_do_Itinerario'] = df['ITINERARY_ID'].astype(int)
        
        # Adicionar coluna COD
        df['COD'] = linha

        return df[['NOME', 'NUM', 'Latitude', 'Longitude', 'SEQ', 'GRUPO', 'SENTIDO', 'TIPO', 'ID_do_Itinerario', 'COD']]

    except requests.RequestException as e:
        logging.error(f"Erro na requisição para a linha {linha}: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar a linha {linha}: {e}")
    return None

# Função para carregar a tabela de ônibus em um DataFrame
def carregar_tabela_para_dataframe(nome_tabela, db='dados_pontos.db'):
    try:
        with sqlite3.connect(db) as conn:
            cursor = conn.cursor()

            # Verificar se a tabela existe
            cursor.execute("""
                SELECT name FROM sqlite_master WHERE type='table' AND name=?;
            """, (nome_tabela,))
            resultado = cursor.fetchone()

            if resultado:
                # Carregar a tabela em um DataFrame
                df = pd.read_sql(f'SELECT * FROM {nome_tabela}', conn)
            else:
                df = pd.DataFrame()  # Retornar DataFrame vazio se a tabela não existir

        return df
    except Exception as e:
        logging.error(f"Erro ao carregar a tabela {nome_tabela}: {e}")
        return pd.DataFrame()

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    try:
        with sqlite3.connect('dados_pontos.db') as conn:
            df_result.to_sql('onibus', conn, if_exists='append', index=False)
            logging.info(f"{len(df_result)} novos pontos gravados no banco de dados.")
    except Exception as e:
        logging.error(f"Erro ao gravar no banco de dados: {e}")

# Função para processar as linhas de ônibus de forma sequencial (sem threads)
def processar_linhas_sequencialmente(linhas_unicas, df_onibus):
    result_list = []

    for linha in linhas_unicas:
        df_result = buscar_e_processar_dados(linha)
        if df_result is not None:
            result_list.append(df_result)

    # Verificar e concatenar resultados
    if result_list:
        df_concatenado = pd.concat(result_list, ignore_index=True)
        logging.info(f"Total de linhas processadas: {len(df_concatenado)}")

        # Inicializar df_novos_pontos
        df_novos_pontos = pd.DataFrame()

        # Verificar se existem novos pontos (baseado em COD)
        if not df_onibus.empty:
            df_novos_pontos = df_concatenado[~df_concatenado['COD'].isin(df_onibus['COD'])]
        else:
            df_novos_pontos = df_concatenado.copy()

        # Gravar novos pontos no banco de dados, se houver
        if not df_novos_pontos.empty:
            gravar_no_banco(df_novos_pontos)
        else:
            logging.info("Nenhum novo ponto para gravar.")

# Inicializar o processamento sequencial
def iniciar_processamento():
    linhas_unicas = range(0, 1000)  # Exemplo de intervalo de linhas
    criar_tabela()  # Garantir que a tabela está criada

    try:
        # Carregar tabela do banco de dados em um DataFrame
        df_onibus = carregar_tabela_para_dataframe('onibus')

        # Processar as linhas de forma sequencial (sem threads)
        processar_linhas_sequencialmente(linhas_unicas, df_onibus)
    except Exception as e:
        logging.error(f"Erro durante o processamento: {e}")
    finally:
        logging.info("Processamento finalizado.")

# Chamar a função principal para iniciar o processamento
if __name__ == "__main__":
    iniciar_processamento()


2024-10-16 23:45:07,833 - ERROR - Erro na requisição para a linha 54: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:45:27,425 - ERROR - Erro na requisição para a linha 86: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Max retries exceeded with url: /getPontosLinha.php?linha=086&c=821f0 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7cdd865d8040>, 'Connection to transporteservico.urbs.curitiba.pr.gov.br timed out. (connect timeout=10)'))
2024-10-16 23:45:49,128 - ERROR - Erro na requisição para a linha 142: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:46:14,321 - ERROR - Erro na requisição para a linha 189: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:46:28,906 - ERROR - Err

In [39]:
df_data_base0 = carregar_tabela_para_dataframe('onibus')
df_data_base0

,NOME,NUM,Latitude,Longitude,SEQ,GRUPO,SENTIDO,TIPO,ID_do_Itinerario,COD
0,"Av. Manoel Ribas, 531 - Mercês",110037,-25.422483,-49.284328,37,,Horário,Novo mobiliário,481,10
1,"Rua Augusto Stresser, 517 - Juvevê",130238,-25.417083,-49.256173,6,,Horário,Novo mobiliário,481,10
2,"Rua Chile, 821 - Rebouças",140051,-25.447454,-49.257707,18,,Horário,Novo mobiliário,481,10
3,"R. Dep. Mário de Barros, 1681 - Centro Cívico",120112,-25.411726,-49.270902,42,,Horário,Novo mobiliário,481,10
4,"R. Alm. Tamandaré, 272 - Alto da Rua Quinze",130268,-25.430345,-49.255576,10,,Horário,Novo mobiliário,481,10
...,...,...,...,...,...,...,...,...,...,...
16292,"Av. Manoel Ribas, 3400 - Cascatinha",190552,-25.414255,-49.308201,19,,Mad. Santa Felicidade (Circular),Novo mobiliário,11053,989
16293,"Av. Manoel Ribas, 7406 - Butiatuvinha",190482,-25.396975,-49.344752,59,,Mad. Santa Felicidade (Circular),Novo mobiliário,11053,989
16294,"R. Visc. de Nacar, 549 - São Francisco",110139,-25.427651,-49.280378,89,,Mad. Santa Felicidade (Circular),Novo mobiliário,11053,989
16295,"Rua Professor Fernando Moreira, 1 - Centro",110448,-25.431305,-49.278688,91,,Mad. Santa Felicidade (Circular),Especial Madrugueiro,11053,989


In [ ]:
df_data_base1 = carregar_tabela_para_dataframe('onibus')
df_data_base1

In [35]:
import sqlite3
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para limpar a tabela onibus no banco de dados
def limpar_tabela_onibus(db='dados_pontos.db'):
    try:
        # Conectar ao banco de dados
        with sqlite3.connect(db) as conn:
            cursor = conn.cursor()
            
            # Executar o comando para limpar a tabela
            cursor.execute("DELETE FROM onibus")
            conn.commit()  # Confirmar as alterações
            
            # Logar o sucesso da operação
            logging.info("Tabela 'onibus' limpa com sucesso.")
    
    except sqlite3.Error as e:
        logging.error(f"Erro ao limpar a tabela: {e}")

# Exemplo de uso
if __name__ == "__main__":
    limpar_tabela_onibus()


2024-10-16 23:33:09,007 - INFO - Tabela 'onibus' limpa com sucesso.
